# import data

In [1]:
import platform
import pandas as pd
from datetime import datetime, timedelta, time
ohlcv_data = pd.read_pickle('/Users/bokie/code/python/CMS/可转债套利/cms/stock_data_min.pkl')
print(ohlcv_data.keys())
for k,v in ohlcv_data.items():
    print(k,' data length: ',len(v))

dict_keys(['123046.SZ', '300587.SZ', '113642.SH', '603185.SH', '110061.SH', '600674.SH', '128081.SZ', '002203.SZ'])
123046.SZ  data length:  31460
300587.SZ  data length:  31460
113642.SH  data length:  13552
603185.SH  data length:  31460
110061.SH  data length:  31460
600674.SH  data length:  31460
128081.SZ  data length:  31460
002203.SZ  data length:  31460


# bond-stock pair

In [ ]:
cbond_pair = ['128081.SZ','002203.SZ']
# cbond_pair = ['123046.SZ','300587.SZ']
# cbond_pair = ['110061.SH','600674.SH']
# cbond_pair = ['113642.SH','603185.SH']

In [ ]:
cbond_pair_price = pd.DataFrame([])
cbond_pair_price = pd.concat([ohlcv_data[cbond_pair[0]]['close'],ohlcv_data[cbond_pair[1]]['close']],axis=1)
cbond_pair_price.columns = cbond_pair
cbond_pair_price = cbond_pair_price.dropna()
cbond_pair_price = cbond_pair_price.loc['2022-03-07 15:00:00':]
# cbond_pair_price.to_excel('cbond_pair_price.xlsx')
# cbond_pair_price.to_csv('cbond_pair_price.csv')

In [ ]:
print(len(cbond_pair_price))
cbond_pair_price.head(10)

# signal calculate

先算cbond_return(日内收益率，相对于昨日收盘价)，再算cbond_position，计算cbond_position中用到的open_threshold和close_threshold分别是价格止盈价格止损信号。时间止损信号则是，每日15点时强制平仓

In [ ]:
cbond_return = pd.DataFrame([])

cbond_preclose = cbond_pair_price.iloc[0,0] # 初始时刻的bond前收盘价
stock_preclose = cbond_pair_price.iloc[0,1] # 初始时刻的stock前收盘价

for index,row in cbond_pair_price.iterrows():
    
    cbond_return.loc[index,cbond_pair[0]] = row[cbond_pair[0]]/cbond_preclose - 1 # 算日内的return
    cbond_return.loc[index,cbond_pair[1]] = row[cbond_pair[1]]/stock_preclose - 1

    if index.time()==time(15,0,0):  #算完日内return，迭代到今日价格的最后一个time bar的时候，将其设为前收盘价
        cbond_preclose = row[cbond_pair[0]]    # 日内return指标用 现价/昨日收盘价 计算
        stock_preclose = row[cbond_pair[1]]
        
# 检查df是否有空值
cbond_return.isnull().sum(axis=0)  # 按列统计
cbond_return.isnull().sum(axis=1) # 按行统计
cbond_return.isnull().sum(axis=1).sum() # 统计整张表

In [ ]:
pd.set_option('display.max_rows', None)
# pair moving trend: diff = cbond reutrn - stock return 
cbond_return['diff'] = cbond_return[cbond_pair[0]] - cbond_return[cbond_pair[1]]

len(cbond_return)

In [ ]:
# # 画出收益率的时间序列图，对diff序列做平稳性检验
# # plot pair moving trend: diff = cbond reutrn - stock return 

# import plotly.graph_objects as go

# fig = go.Figure(data=[
#     go.Scatter(name=cbond_pair[0], x=cbond_return.index, y=cbond_return[cbond_pair[0]]),
#     go.Scatter(name=cbond_pair[1], x=cbond_return.index, y=cbond_return[cbond_pair[1]]),
#     go.Scatter(name='diff', x=cbond_return.index, y=cbond_return['diff']),
# ])
# fig.show()


# # test stationary
# from statsmodels.tsa.stattools import adfuller
# if adfuller(cbond_return['diff'])[1] < 0.001:
#     print("This cbond-stock pair is stationary!")
# else:
#     print("This cbond-stock pair is not stationary!")

In [ ]:
open_threshold = 0.01
# open_threshold = 0.008
close_threshold = 0.001
open_flag = 0
cbond_position = pd.DataFrame([])
cbond_position.index = cbond_return.index
for index,row in cbond_return.iterrows():
    if abs(row['diff']) > open_threshold and open_flag==0:  # diff大于开仓阈值，并且无仓位就开仓
        open_flag = 1
        if row['diff'] > 0:  # diff 绝对值大于0.01并且diff是正数，说明bond偏高了，做空之，stock则反向开仓
            cbond_position.loc[index,cbond_pair[0]] = -1*open_flag  #short bond, long stock
            cbond_position.loc[index,cbond_pair[1]] = open_flag
        else:                # diff 绝对值大于0.01并且diff是负数，说明bond偏低了，做多之，stock则反向开仓
            cbond_position.loc[index,cbond_pair[0]] = open_flag     #long bond, short stock
            cbond_position.loc[index,cbond_pair[1]] = -1*open_flag
    
    elif abs(row['diff']) < close_threshold and open_flag==1: # diff小于平仓阈值，并且有仓位就平仓
        open_flag = 0   # diff 绝对值小于阈值时，认为股债的偏移效应减弱，走势收敛，此时平仓
        cbond_position.loc[index,cbond_pair[0]] = open_flag
        cbond_position.loc[index,cbond_pair[1]] = open_flag
    
    else: # diff介于开仓阈值和平仓阈值之间，并且diff大于0，short bond long stock,小于0则short stock long bond
        if row['diff'] > 0:
            cbond_position.loc[index,cbond_pair[0]] = -1*open_flag
            cbond_position.loc[index,cbond_pair[1]] = open_flag
        else:
            cbond_position.loc[index,cbond_pair[0]] = open_flag
            cbond_position.loc[index,cbond_pair[1]] = -1*open_flag

    if index.time()==time(15,0,0): # 每日收盘时必须平仓（时间止损信号）
        open_flag = 0 
        cbond_position.loc[index,cbond_pair[0]] = open_flag
        cbond_position.loc[index,cbond_pair[1]] = open_flag
# cbond_position.to_csv('cbond_position.csv')

In [ ]:
df_signal = pd.concat([cbond_pair_price,cbond_return,cbond_position],axis=1)
df_signal.columns = ['bond_price','stock_price','bond_rtn','stock_rtn','diff','bond_pos','stock_pos']
df_signal.to_csv('df_signal.csv')


In [ ]:
trade_times = 0
trade_occur = []
for i in range(len(df_signal)-1):
    # print(df_signal.values[i])
    # if cbond_position.values[i][-1] != cbond_position.values[i+1][-1]:
    if df_signal.values[i][-1] != df_signal.values[i+1][-1]:
        trade_times += 1
        trade_occur.append(df_signal.index.tolist()[i])
        trade_occur.append(df_signal.index.tolist()[i+1])
    # assert i<100
print('trade_times: ',trade_times)

In [ ]:
# diff = bond return - stock return
len(trade_occur)
df_signal.loc[trade_occur]
# df_signal.loc[time(15,0,0)]
# df_signal

In [ ]:
# for index,row in df_signal.iterrows():
#     print(row)
#     assert False

# cbond_rtreturn = (cbond_pair_price.pct_change()+1).fillna(0)
# cbond_rtreturn

# backtest

In [ ]:
# 用来记录交易流水的表格,一边遍历df_signal，一边填入内容到流水表格里
import numpy as np
cols = ['balance','signal','bond price','stock price','bond hold','stock hold','commission','tax']
df_trade = pd.DataFrame(
    data = [[0,np.nan,np.nan,np.nan,0,0,0,0]],
    columns=cols
    ) 

balance_last = 0 # 现金账户初始余额

bond_hold = 0 # 债券持有数量
stock_hold = 0 # 股票持有数量

commission_rate = 2/10000 # 佣金买卖万二
tax_rate = 1/1000 #

stock_volume = 100/9.69

for i in range(len(df_signal)-1):
    if df_signal.values[i][-1] != df_signal.values[i+1][-1]:
        index = df_signal.index[i+1]

    #  ['bond_price','stock_price','bond_rtn','stock_rtn','diff','bond_pos','stock_pos']
        signal = str(int(df_signal.values[i+1][-2]))+str(int(df_signal.values[i+1][-1]))
        

        bond_price = df_signal.values[i+1][0] 

        stock_price = df_signal.values[i+1][1]

        commission = (bond_price + stock_price * stock_volume) * commission_rate

        if signal == '00':
            tax = stock_price * tax_rate * stock_volume
        else:
            tax = 0

        if signal == '00':
            balance_last += bond_hold*bond_price
            balance_last += stock_hold*stock_price
            
            bond_hold = 0
            stock_hold = 0
            

        if signal == '1-1':
            bond_hold += 1
            stock_hold += (-1 * stock_volume)
            # stock_hold += stock_volume
            balance_last -= bond_price
            balance_last += stock_price * stock_volume


        if signal == '-11':
            bond_hold += -1
            stock_hold += (1 * stock_volume)
            # stock_hold += stock_volume
            balance_last += bond_price
            balance_last -= stock_price * stock_volume
        

        balance = balance_last
        
        
        line = [balance,signal,bond_price,stock_price,bond_hold,stock_hold,commission,tax]
        # df_trade.loc[index,['balance','asset','signal','bond price','stock price','commission','tax']] = line
        # df_trade.loc[index,cols] = [0,0,np.nan,np.nan,np.nan,0,0,0,0]
        df_trade.loc[index,cols] = line

       
# df_trade

# result

In [ ]:
df_trade['net value'] = df_trade['balance'] + \
                        df_trade['bond price']*df_trade['bond hold'] + \
                        df_trade['stock price']*df_trade['stock hold'] - \
                        df_trade['commission'] - \
                        df_trade['tax']

In [ ]:
df_trade

In [ ]:
df_nv = df_trade[['net value']].iloc[1:]
df_nv['time'] = df_nv.index
df_tmp = df_signal[['diff']]
df_tmp['time'] = df_tmp.index
# df_signal['time'] = df_signal.index
df_nv_full =  pd.merge(df_nv,df_tmp,how='outer').sort_values(by='time').fillna(method='bfill')
df_nv_full.index = df_tmp.index
df_nv_full = df_nv_full[['net value']]
# # df_signal

In [ ]:
df_nv_full.loc[df_nv_full.index[0]:df_trade.index[1]] = [0]
df_nv_full.plot(figsize=(12,6))

In [ ]:
# df_nv_full

In [ ]:
# df_nv_full['return'] = df_nv_full['net value'].pct_change()
# df_nv_full

In [ ]:
# df_nv_full.loc[df_trade.index[1]:].plot(figsize=(12,6))

In [ ]:
# balance_final = df_trade['balance'].values[-1] - df_trade['commission'].sum() + df_trade['tax'].sum()
# balance_final
# df_balance = df_trade[['balance']].iloc[1:]
# df_balance['time'] = df_balance.index
# df_tmp = df_signal[['diff']]
# df_tmp['time'] = df_tmp.index
# # df_signal['time'] = df_signal.index
# df_balance_full =  pd.merge(df_balance,df_tmp,how='outer').sort_values(by='time').fillna(method='bfill')
# df_balance_full.index = df_tmp.index
# df_balance_full = df_balance_full[['balance']]
# # df_signal
# df_balance_full.loc[df_trade.index[2]:].plot(figsize=(12,6))